In [ ]:
#!pip install basicsr
!git clone https://github.com/XPixelGroup/BasicSR.git

In [ ]:
%cd BasicSR
import cv2
import torch
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
import lpips
from basicsr.metrics import calculate_psnr, calculate_ssim
from basicsr.metrics.psnr_ssim import calculate_psnr_pt, calculate_ssim_pt
from basicsr.utils import img2tensor

def test_folder(folder1, folder2, crop_border, test_y_channel=False):
    files1 = sorted(os.listdir(folder1))
    files2 = sorted(os.listdir(folder2))

    assert len(files1) == len(files2), "Folders should have the same number of files"

    # Initialize LPIPS models
    loss_fn_alex = lpips.LPIPS(net='alex').cuda()
    loss_fn_vgg = lpips.LPIPS(net='vgg').cuda()

    total_scores = {

        'psnr_pth_gpu': 0, 'ssim_pth_gpu': 0,
        'lpips_alex': 0, 'lpips_vgg': 0
    }

    for file1, file2 in zip(files1, files2):
        img_path = os.path.join(folder1, file1)
        img_path2 = os.path.join(folder2, file2)
        scores = test_image_pair(img_path, img_path2, crop_border, test_y_channel, loss_fn_alex, loss_fn_vgg)
        for key in total_scores.keys():
            total_scores[key] += scores[key]

    num_images = len(files1)
    avg_scores = {key: total / num_images for key, total in total_scores.items()}

    print('\nAverage Scores:')
    for key, score in avg_scores.items():
        print(f'{key}: {score:.6f}')

def test_image_pair(img_path, img_path2, crop_border, test_y_channel, loss_fn_alex, loss_fn_vgg):
    img = cv2.imread(img_path, cv2.IMREAD_UNCHANGED)
    img2 = cv2.imread(img_path2, cv2.IMREAD_UNCHANGED)

    # Check if dimensions match, and if not, rotate the second image
    if img.shape[:2] != img2.shape[:2]:
        print("Rotate")
        # Rotate img2 by 90 degrees to match img's orientation
        img2 = cv2.rotate(img2, cv2.ROTATE_90_CLOCKWISE)


    # PyTorch (CPU) calculations
    img_pth = img2tensor(img / 255., bgr2rgb=True, float32=True).unsqueeze_(0)
    img2_pth = img2tensor(img2 / 255., bgr2rgb=True, float32=True).unsqueeze_(0)


    # PyTorch (GPU) calculations
    psnr_pth_gpu, ssim_pth_gpu, lpips_alex, lpips_vgg = 0, 0, 0, 0
  #  if torch.cuda.is_available():
    img_pth_gpu = img_pth.cuda()
    img2_pth_gpu = img2_pth.cuda()

    psnr_pth_gpu = calculate_psnr_pt(img_pth_gpu, img2_pth_gpu, crop_border=crop_border, test_y_channel=test_y_channel)
    ssim_pth_gpu = calculate_ssim_pt(img_pth_gpu, img2_pth_gpu, crop_border=crop_border, test_y_channel=test_y_channel)

    # LPIPS calculations
    lpips_alex = loss_fn_alex(img_pth_gpu, img2_pth_gpu)
    lpips_vgg = loss_fn_vgg(img_pth_gpu, img2_pth_gpu)

    return {

        'psnr_pth_gpu': psnr_pth_gpu[0], 'ssim_pth_gpu': ssim_pth_gpu[0],
        'lpips_alex': lpips_alex.item(), 'lpips_vgg': lpips_vgg.item()
    }


if __name__ == '__main__':
    path_base = '/data/Image_restoration/Datasets/'

    folder1 = '/data/Image_restoration/Datasets/RS_AID_data/AID-dataset/test/HR'#AID

    folder2 = '/data/Image_restoration/Datasets/RS_AID_data/AID-dataset/test/out_x4'#AID
   
    test_folder(folder1, folder2, crop_border=4, test_y_channel=True)
 